In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import boto3
import json
import itertools
import pandas as pd
import numpy as np

<IPython.core.display.Javascript object>

In [3]:
cs_clnt = boto3.client("cloudsearch", region_name="ap-southeast-2")

<IPython.core.display.Javascript object>

In [4]:
domain_name = "test"

<IPython.core.display.Javascript object>

In [5]:
response = cs_clnt.describe_domains(DomainNames=[domain_name])
doc_ep = response["DomainStatusList"][0]["DocService"]["Endpoint"]
search_ep = response["DomainStatusList"][0]["SearchService"]["Endpoint"]
print("doc_ep: ", doc_ep)
print("search_ep: ", search_ep)

doc_ep:  doc-test-vbvmblazfsw7s6qe4anojm23iy.ap-southeast-2.cloudsearch.amazonaws.com
search_ep:  search-test-vbvmblazfsw7s6qe4anojm23iy.ap-southeast-2.cloudsearch.amazonaws.com


<IPython.core.display.Javascript object>

In [6]:
# doc_client = boto3.client("cloudsearchdomain", endpoint_url=f"https://{doc_ep}")

<IPython.core.display.Javascript object>

In [7]:
src_clnt = boto3.client("cloudsearchdomain", endpoint_url=f"https://{search_ep}")

<IPython.core.display.Javascript object>

In [8]:
def parse_response(response):
    df = pd.DataFrame()

    nhits = len(response["hits"]["hit"])
    print("nhits: ", nhits)
    if nhits == 0:
        return df

    for i in range(nhits):
        row = response["hits"]["hit"][i]["fields"]
        df = pd.concat([df, pd.DataFrame([row])])

    for col in df.columns:
        df[col] = df[col].str.join(",")
        # df[col] = list(itertools.chain(*df[col]))

    return df

<IPython.core.display.Javascript object>

### Search Engine: simple
- text and text-array
- phrases, individual terms, and prefixes

In [9]:
query = "The Hunger Games"

response = src_clnt.search(query=query, queryParser="simple", size=100)
results_df = parse_response(response)
results_df.head(5)

nhits:  4


,directors,release_date,genres,image_url,plot,title,rank,running_time_secs,actors,year,rating
0,Francis Lawrence,2013-11-11T00:00:00Z,"Action,Adventure,Sci-Fi,Thriller",http://ia.media-imdb.com/images/M/MV5BMTAyMjQ3...,Katniss Everdeen and Peeta Mellark become targ...,The Hunger Games: Catching Fire,4,8760,"Jennifer Lawrence,Josh Hutcherson,Liam Hemsworth",2013,NaN
0,Gary Ross,2012-03-12T00:00:00Z,"Adventure,Sci-Fi,Thriller",http://ia.media-imdb.com/images/M/MV5BMjA4NDg3...,Katniss Everdeen voluntarily takes her younger...,The Hunger Games,82,8520,"Jennifer Lawrence,Josh Hutcherson,Liam Hemsworth",2012,7.2
0,Francis Lawrence,2014-11-20T00:00:00Z,"Action,Adventure,Drama,Sci-Fi",NaN,Katniss Everdeen reluctantly becomes the symbo...,The Hunger Games: Mockingjay - Part 1,688,NaN,"Jennifer Lawrence,Liam Hemsworth,Natalie Dormer",2014,NaN
0,Francis Lawrence,2015-11-20T00:00:00Z,"Action,Adventure,Drama,Sci-Fi,Thriller",NaN,Katniss Everdeen and wayward Peeta Mellark att...,The Hunger Games: Mockingjay - Part 2,2103,NaN,"Jennifer Lawrence,Liam Hemsworth,Natalie Dormer",2015,NaN


<IPython.core.display.Javascript object>

In [10]:
query = "The Hunger Games | Transformers"

response = src_clnt.search(query=query, queryParser="simple", size=100)
results_df = parse_response(response)
results_df.head(5)

nhits:  36


,directors,release_date,genres,image_url,plot,title,rank,running_time_secs,actors,year,rating
0,Francis Lawrence,2013-11-11T00:00:00Z,"Action,Adventure,Sci-Fi,Thriller",http://ia.media-imdb.com/images/M/MV5BMTAyMjQ3...,Katniss Everdeen and Peeta Mellark become targ...,The Hunger Games: Catching Fire,4,8760,"Jennifer Lawrence,Josh Hutcherson,Liam Hemsworth",2013,NaN
0,Gary Ross,2012-03-12T00:00:00Z,"Adventure,Sci-Fi,Thriller",http://ia.media-imdb.com/images/M/MV5BMjA4NDg3...,Katniss Everdeen voluntarily takes her younger...,The Hunger Games,82,8520,"Jennifer Lawrence,Josh Hutcherson,Liam Hemsworth",2012,7.2
0,Michael Bay,2009-06-19T00:00:00Z,"Action,Adventure,Sci-Fi,Thriller",http://ia.media-imdb.com/images/M/MV5BNjk4OTcz...,Sam Witwicky leaves the Autobots behind for a ...,Transformers: Revenge of the Fallen,804,9000,"Shia LaBeouf,Megan Fox,Josh Duhamel",2009,5.9
0,Michael Bay,2007-06-12T00:00:00Z,"Action,Adventure,Sci-Fi,Thriller",http://ia.media-imdb.com/images/M/MV5BMTQwNjU5...,An ancient struggle between two Cybertronian r...,Transformers,434,8640,"Shia LaBeouf,Megan Fox,Josh Duhamel",2007,7.1
0,Francis Lawrence,2014-11-20T00:00:00Z,"Action,Adventure,Drama,Sci-Fi",NaN,Katniss Everdeen reluctantly becomes the symbo...,The Hunger Games: Mockingjay - Part 1,688,NaN,"Jennifer Lawrence,Liam Hemsworth,Natalie Dormer",2014,NaN


<IPython.core.display.Javascript object>

In [11]:
query = "The Hunger Games | Transformers"

response = src_clnt.search(
    query=query, queryParser="simple", queryOptions='{"fields":["title"]}', size=100
)
results_df = parse_response(response)
results_df.head(5)

nhits:  9


,directors,release_date,rating,genres,image_url,plot,title,rank,running_time_secs,actors,year
0,Gary Ross,2012-03-12T00:00:00Z,7.2,"Adventure,Sci-Fi,Thriller",http://ia.media-imdb.com/images/M/MV5BMjA4NDg3...,Katniss Everdeen voluntarily takes her younger...,The Hunger Games,82,8520,"Jennifer Lawrence,Josh Hutcherson,Liam Hemsworth",2012
0,Francis Lawrence,2013-11-11T00:00:00Z,NaN,"Action,Adventure,Sci-Fi,Thriller",http://ia.media-imdb.com/images/M/MV5BMTAyMjQ3...,Katniss Everdeen and Peeta Mellark become targ...,The Hunger Games: Catching Fire,4,8760,"Jennifer Lawrence,Josh Hutcherson,Liam Hemsworth",2013
0,Michael Bay,2007-06-12T00:00:00Z,7.1,"Action,Adventure,Sci-Fi,Thriller",http://ia.media-imdb.com/images/M/MV5BMTQwNjU5...,An ancient struggle between two Cybertronian r...,Transformers,434,8640,"Shia LaBeouf,Megan Fox,Josh Duhamel",2007
0,Francis Lawrence,2014-11-20T00:00:00Z,NaN,"Action,Adventure,Drama,Sci-Fi",NaN,Katniss Everdeen reluctantly becomes the symbo...,The Hunger Games: Mockingjay - Part 1,688,NaN,"Jennifer Lawrence,Liam Hemsworth,Natalie Dormer",2014
0,Francis Lawrence,2015-11-20T00:00:00Z,NaN,"Action,Adventure,Drama,Sci-Fi,Thriller",NaN,Katniss Everdeen and wayward Peeta Mellark att...,The Hunger Games: Mockingjay - Part 2,2103,NaN,"Jennifer Lawrence,Liam Hemsworth,Natalie Dormer",2015


<IPython.core.display.Javascript object>

In [12]:
query = "The Hunger Games & Transformers"

response = src_clnt.search(
    query=query, queryParser="simple", queryOptions='{"fields":["title"]}', size=100
)
results_df = parse_response(response)
# results_df.head(5)

nhits:  0


<IPython.core.display.Javascript object>

#### prefix

- document id field: _id 

In [13]:
query = "tt222*"
response = src_clnt.search(
    query=query, queryParser="simple", queryOptions='{"fields":["_id"]}', size=100
)
results_df = parse_response(response)
results_df.head(5)

nhits:  7


,directors,release_date,rating,genres,image_url,plot,title,rank,running_time_secs,actors,year
0,Joseph Gordon-Levitt,2013-01-18T00:00:00Z,7.4,"Comedy,Drama",http://ia.media-imdb.com/images/M/MV5BMTQxNTc3...,"A New Jersey guy dedicated to his family, frie...",Don Jon,1,5400,"Joseph Gordon-Levitt,Scarlett Johansson,Julian...",2013
0,James Wan,2013-09-13T00:00:00Z,7.1,"Horror,Thriller",http://ia.media-imdb.com/images/M/MV5BMTg0OTA5...,The haunted Lambert family seeks to uncover th...,Insidious: Chapter 2,7,6360,"Patrick Wilson,Rose Byrne,Barbara Hershey",2013
0,Logan Miller,2013-01-24T00:00:00Z,6.0,"Thriller,Western",http://ia.media-imdb.com/images/M/MV5BMjM1NzUy...,"In the late 1800s, a fanatical religious leade...",Sweetwater,1398,5700,"Ed Harris,January Jones,Jason Isaacs",2013
0,Vikramaditya Motwane,2013-07-04T00:00:00Z,7.4,"Drama,Romance",http://ia.media-imdb.com/images/M/MV5BOTUwNTYy...,NaN,Lootera,2463,8160,"Ranveer Singh,Sonakshi Sinha,Adil Hussain",2013
0,Ivan Reitman,2014-01-01T00:00:00Z,NaN,"Drama,Sport",http://ia.media-imdb.com/images/M/MV5BMjIyNTg5...,The General Manager of the Cleveland Browns st...,Draft Day,2906,NaN,"Jennifer Garner,Terry Crews,Ellen Burstyn",2014


<IPython.core.display.Javascript object>

### Search Engine: Lucene
- text and text-array
- phrases, individual terms, and prefixes

#### prefix & postfix

In [14]:
query = "tt222*9"
response = src_clnt.search(
    query=query, queryParser="lucene", queryOptions='{"fields":["_id"]}', size=100
)
results_df = parse_response(response)
results_df.head(5)

nhits:  2


,directors,release_date,rating,genres,image_url,plot,title,rank,running_time_secs,actors,year
0,Joseph Gordon-Levitt,2013-01-18T00:00:00Z,7.4,"Comedy,Drama",http://ia.media-imdb.com/images/M/MV5BMTQxNTc3...,"A New Jersey guy dedicated to his family, frie...",Don Jon,1,5400,"Joseph Gordon-Levitt,Scarlett Johansson,Julian...",2013
0,Catherine Hardwicke,2013-09-13T00:00:00Z,5.3,Thriller,http://ia.media-imdb.com/images/M/MV5BNjQ4MTQ0...,"A young singer/songwriter, despite being marri...",Plush,3150,NaN,"Emily Browning,Cam Gigandet,Xavier Samuel",2013


<IPython.core.display.Javascript object>

#### prefix & postfix with fixed number of chars inbetween

In [15]:
query = "tt222?9"
response = src_clnt.search(
    query=query, queryParser="lucene", queryOptions='{"fields":["_id"]}', size=100
)
results_df = parse_response(response)
results_df.head(5)

nhits:  0


""


<IPython.core.display.Javascript object>

### Search Engine: Structured


#### Numbers
- double, double-array, int, and int-array. 

In [16]:
query = "year:2016"
query = "(term field=year 2016)"

response = src_clnt.search(query=query, queryParser="structured", size=100)
results_df = parse_response(response)
results_df.head(5)

nhits:  3


,directors,genres,plot,title,rank,actors,year,release_date
0,"Joachim Rønning,Espen Sandberg","Action,Adventure,Comedy,Fantasy",The fifth installment of the blockbuster franc...,Pirates of the Caribbean: Dead Men Tell No Tales,384,Johnny Depp,2016,NaN
0,James Cameron,"Action,Adventure,Fantasy,Sci-Fi",A sequel to the 2009 global phenomenon.,Avatar 2,909,"Zoe Saldana,Sigourney Weaver,Sam Worthington",2016,2016-12-01T00:00:00Z
0,Greg Berlanti,"Action,Adventure,Drama,Fantasy,Sci-Fi",NaN,The Flash,2281,NaN,2016,2016-01-01T00:00:00Z


<IPython.core.display.Javascript object>

In [17]:
query = "(range field=year [2016,})"

response = src_clnt.search(query=query, queryParser="structured", size=100)
results_df = parse_response(response)
results_df.head(5)

nhits:  5


,directors,genres,plot,title,rank,actors,year,release_date
0,"Joachim Rønning,Espen Sandberg","Action,Adventure,Comedy,Fantasy",The fifth installment of the blockbuster franc...,Pirates of the Caribbean: Dead Men Tell No Tales,384,Johnny Depp,2016,NaN
0,James Cameron,"Action,Adventure,Fantasy,Sci-Fi",A sequel to the 2009 global phenomenon.,Avatar 2,909,"Zoe Saldana,Sigourney Weaver,Sam Worthington",2016,2016-12-01T00:00:00Z
0,NaN,"Action,Adventure,Fantasy,Sci-Fi",The world's greatest heroes are assembled to f...,Justice League,1295,NaN,2017,2017-01-01T00:00:00Z
0,Greg Berlanti,"Action,Adventure,Drama,Fantasy,Sci-Fi",NaN,The Flash,2281,NaN,2016,2016-01-01T00:00:00Z
0,Patrick Lussier,Horror,The plot is unknown at this time.,Halloween III,4896,NaN,2018,NaN


<IPython.core.display.Javascript object>

In [18]:
query = "(range field=year [2016,2017])"

response = src_clnt.search(query=query, queryParser="structured", size=100)
results_df = parse_response(response)
results_df.head(5)

nhits:  4


,directors,genres,plot,title,rank,actors,year,release_date
0,"Joachim Rønning,Espen Sandberg","Action,Adventure,Comedy,Fantasy",The fifth installment of the blockbuster franc...,Pirates of the Caribbean: Dead Men Tell No Tales,384,Johnny Depp,2016,NaN
0,James Cameron,"Action,Adventure,Fantasy,Sci-Fi",A sequel to the 2009 global phenomenon.,Avatar 2,909,"Zoe Saldana,Sigourney Weaver,Sam Worthington",2016,2016-12-01T00:00:00Z
0,NaN,"Action,Adventure,Fantasy,Sci-Fi",The world's greatest heroes are assembled to f...,Justice League,1295,NaN,2017,2017-01-01T00:00:00Z
0,Greg Berlanti,"Action,Adventure,Drama,Fantasy,Sci-Fi",NaN,The Flash,2281,NaN,2016,2016-01-01T00:00:00Z


<IPython.core.display.Javascript object>

#### Dates and Times
- date and date-array

In [19]:
query = "release_date:'2001-12-25T00:00:00Z'"
query = "(term field=release_date '2001-12-25T00:00:00Z')"

response = src_clnt.search(query=query, queryParser="structured", size=100)
results_df = parse_response(response)
results_df.head(5)

nhits:  1


,directors,release_date,rating,genres,image_url,title,rank,running_time_secs,actors,year
0,James Mangold,2001-12-25T00:00:00Z,6.2,"Comedy,Fantasy,Romance",http://ia.media-imdb.com/images/M/MV5BODU1MzU0...,Kate & Leopold,3474,7080,"Meg Ryan,Hugh Jackman,Liev Schreiber",2001


<IPython.core.display.Javascript object>

#### Compound queries
- and, or, not

In [25]:
query = "('star')"

# query  = "(and (phrase field=title 'star') (phrase field=actors 'nick'))"
# query  = "(and (term field=title 'star') (term field=actors 'nick'))"

response = src_clnt.search(
    query=query,
    queryParser="structured",
)
results_df = parse_response(response)
results_df.head(5)

SearchException: An error occurred (SearchException) when calling the Search operation: Error Count: 1; [1] Syntax error in query: invalid sequence of tokens near (() at position (0).

<IPython.core.display.Javascript object>

In [20]:
query = "(and title:'star' actors:'nick')"

# query  = "(and (phrase field=title 'star') (phrase field=actors 'nick'))"
# query  = "(and (term field=title 'star') (term field=actors 'nick'))"

response = src_clnt.search(
    query=query,
    queryParser="structured",
)
results_df = parse_response(response)
results_df.head(5)

nhits:  1


,directors,release_date,rating,genres,image_url,plot,title,rank,running_time_secs,actors,year
0,Tom Brady,2011-05-27T00:00:00Z,3.1,Comedy,http://ia.media-imdb.com/images/M/MV5BMjIzMjY4...,A kid from the Midwest moves out to Hollywood ...,Bucky Larson: Born to Be a Star,3720,5820,"Nick Swardson,Don Johnson,Christina Ricci",2011


<IPython.core.display.Javascript object>

In [21]:
query = "(and 'star' (not actors:'Zachary Quinto'))"

response = src_clnt.search(
    query=query,
    queryParser="structured",
)
results_df = parse_response(response)
results_df.head(5)



nhits:  10


,directors,release_date,rating,genres,image_url,plot,title,rank,running_time_secs,actors,year
0,Tom Brady,2011-05-27T00:00:00Z,3.1,Comedy,http://ia.media-imdb.com/images/M/MV5BMjIzMjY4...,A kid from the Midwest moves out to Hollywood ...,Bucky Larson: Born to Be a Star,3720,5820,"Nick Swardson,Don Johnson,Christina Ricci",2011
0,Simon J. Smith,2015-03-27T00:00:00Z,NaN,"Animation,Adventure,Comedy,Family",NaN,The Penguins will star in the film. A spin-off...,The Penguins of Madagascar,4309,NaN,"Benedict Cumberbatch,Ben Stiller,Jada Pinkett ...",2015
0,Richard Marquand,1983-05-25T00:00:00Z,8.4,"Action,Adventure,Fantasy,Sci-Fi",http://ia.media-imdb.com/images/M/MV5BMjE5MTM3...,After rescuing Han Solo from the palace of Jab...,Star Wars: Episode VI - Return of the Jedi,782,8040,"Mark Hamill,Harrison Ford,Carrie Fisher",1983
0,Spike Jonze,1999-09-02T00:00:00Z,7.8,"Comedy,Fantasy,Sci-Fi",http://ia.media-imdb.com/images/M/MV5BMTUyMjI4...,A puppeteer discovers a portal that leads lite...,Being John Malkovich,1506,6720,"John Cusack,Cameron Diaz,Catherine Keener",1999
0,Matt Hish,2013-09-01T00:00:00Z,3.2,Comedy,http://ia.media-imdb.com/images/M/MV5BMzY3MTc1...,Some people drink to forget. Rock stars drink ...,The Blackout,1616,5400,"Michael Graziadei,Scott Rosa,Flood Reed",2013


<IPython.core.display.Javascript object>

In [22]:
query = "(and (phrase field='title' 'star wars') (not (range field=year {,2000})))"

response = src_clnt.search(
    query=query,
    queryParser="structured",
)
results_df = parse_response(response)
results_df.head(5)

nhits:  3


,directors,release_date,genres,plot,title,rank,year,actors,rating,image_url,running_time_secs
0,J.J. Abrams,2015-01-01T00:00:00Z,"Action,Adventure,Fantasy,Sci-Fi",A continuation of the saga created by George L...,Star Wars: Episode VII,168,2015,"Mark Hamill,Harrison Ford,Carrie Fisher",NaN,NaN,NaN
0,George Lucas,2005-05-15T00:00:00Z,"Action,Adventure,Fantasy,Sci-Fi",After three years of fighting in the Clone War...,Star Wars: Episode III - Revenge of the Sith,371,2005,"Hayden Christensen,Natalie Portman,Ewan McGregor",7.7,http://ia.media-imdb.com/images/M/MV5BNTc4MTc3...,8400
0,George Lucas,2002-05-16T00:00:00Z,"Action,Adventure,Fantasy,Sci-Fi","Ten years later, Anakin Skywalker shares a for...",Star Wars: Episode II - Attack of the Clones,922,2002,"Hayden Christensen,Natalie Portman,Ewan McGregor",6.7,http://ia.media-imdb.com/images/M/MV5BMTY5MjI5...,8520


<IPython.core.display.Javascript object>

In [31]:
query = "(and title:'star wars' year: {,2000])"


# query = "(and title:'star' actors:'nick')"


response = src_clnt.search(
    query=query,
    queryParser="structured",
)
results_df = parse_response(response)
results_df.head(5)

nhits:  4


,directors,release_date,rating,genres,image_url,plot,title,rank,running_time_secs,actors,year
0,George Lucas,1977-05-25T00:00:00Z,8.7,"Action,Adventure,Fantasy,Sci-Fi",http://ia.media-imdb.com/images/M/MV5BMTU4NTcz...,Luke Skywalker joins forces with a Jedi Knight...,Star Wars,226,7260,"Mark Hamill,Harrison Ford,Carrie Fisher",1977
0,George Lucas,1999-05-19T00:00:00Z,6.5,"Action,Adventure,Fantasy,Sci-Fi",http://ia.media-imdb.com/images/M/MV5BMTQ4NjEw...,Two Jedi Knights escape a hostile blockade to ...,Star Wars: Episode I - The Phantom Menace,469,8160,"Ewan McGregor,Liam Neeson,Natalie Portman",1999
0,Irvin Kershner,1980-05-17T00:00:00Z,8.8,"Action,Adventure,Sci-Fi",http://ia.media-imdb.com/images/M/MV5BMjE2MzQw...,After the rebels have been brutally overpowere...,Star Wars: Episode V - The Empire Strikes Back,651,7440,"Mark Hamill,Harrison Ford,Carrie Fisher",1980
0,Richard Marquand,1983-05-25T00:00:00Z,8.4,"Action,Adventure,Fantasy,Sci-Fi",http://ia.media-imdb.com/images/M/MV5BMjE5MTM3...,After rescuing Han Solo from the palace of Jab...,Star Wars: Episode VI - Return of the Jedi,782,8040,"Mark Hamill,Harrison Ford,Carrie Fisher",1983


<IPython.core.display.Javascript object>

In [23]:
query = "(and title:'star' (or actors:'Harrison Ford' actors:'William Shatner') (not actors:'Zachary Quinto'))"

response = src_clnt.search(
    query=query,
    queryParser="structured",
)
results_df = parse_response(response)
results_df.head(5)

nhits:  10


,directors,release_date,rating,genres,image_url,plot,title,rank,running_time_secs,actors,year
0,George Lucas,1977-05-25T00:00:00Z,8.7,"Action,Adventure,Fantasy,Sci-Fi",http://ia.media-imdb.com/images/M/MV5BMTU4NTcz...,Luke Skywalker joins forces with a Jedi Knight...,Star Wars,226,7260,"Mark Hamill,Harrison Ford,Carrie Fisher",1977
0,Nicholas Meyer,1982-06-04T00:00:00Z,7.7,"Action,Adventure,Sci-Fi,Thriller",http://ia.media-imdb.com/images/M/MV5BMTcwNjc5...,"With the aid of the Enterprise crew, Admiral K...",Star Trek: The Wrath of Khan,367,6780,"William Shatner,Leonard Nimoy,DeForest Kelley",1982
0,Robert Wise,1979-12-06T00:00:00Z,6.3,"Adventure,Mystery,Sci-Fi",http://ia.media-imdb.com/images/M/MV5BNzE4MTY1...,When a destructive space entity is spotted app...,Star Trek: The Motion Picture,1837,7920,"William Shatner,Leonard Nimoy,DeForest Kelley",1979
0,David Carson,1994-11-17T00:00:00Z,6.5,"Action,Adventure,Sci-Fi,Thriller",http://ia.media-imdb.com/images/M/MV5BOTMyODky...,"Captain Picard, with the help of supposedly de...",Star Trek: Generations,2041,7080,"Patrick Stewart,William Shatner,Malcolm McDowell",1994
0,J.J. Abrams,2015-01-01T00:00:00Z,NaN,"Action,Adventure,Fantasy,Sci-Fi",NaN,A continuation of the saga created by George L...,Star Wars: Episode VII,168,NaN,"Mark Hamill,Harrison Ford,Carrie Fisher",2015


<IPython.core.display.Javascript object>

In [24]:
query = "(and (not field=genres 'Sci-Fi')(or (term field=title boost=2 'star')(term field=plot 'star')))"

response = src_clnt.search(
    query=query,
    queryParser="structured",
)
results_df = parse_response(response)
results_df.head(5)

nhits:  10


,directors,release_date,rating,genres,image_url,plot,title,rank,running_time_secs,actors,year
0,Tom Brady,2011-05-27T00:00:00Z,3.1,Comedy,http://ia.media-imdb.com/images/M/MV5BMjIzMjY4...,A kid from the Midwest moves out to Hollywood ...,Bucky Larson: Born to Be a Star,3720,5820,"Nick Swardson,Don Johnson,Christina Ricci",2011
0,John Sayles,1996-06-21T00:00:00Z,7.5,"Drama,Mystery,Romance",http://ia.media-imdb.com/images/M/MV5BMTU3OTI2...,When the skeleton of his murdered predecessor ...,Lone Star,1871,8100,"Chris Cooper,Elizabeth Peña,Stephen Mendillo",1996
0,Stephen Herek,2001-09-04T00:00:00Z,5.9,"Comedy,Drama,Music",http://ia.media-imdb.com/images/M/MV5BMjE4NTYy...,Lead singer of a tribute band becomes lead sin...,Rock Star,2509,6300,"Mark Wahlberg,Jennifer Aniston,Dominic West",2001
0,Jane Campion,2009-05-15T00:00:00Z,6.9,"Biography,Drama,Romance",http://ia.media-imdb.com/images/M/MV5BMTg0NjEw...,The three-year romance between 19th century po...,Bright Star,2571,7140,"Abbie Cornish,Ben Whishaw,Paul Schneider",2009
0,David Cronenberg,2014-09-25T00:00:00Z,NaN,Drama,NaN,Complex look at Hollywood and what it reveals ...,Maps to the Stars,3873,NaN,"Julianne Moore,Robert Pattinson,Mia Wasikowska",2014


<IPython.core.display.Javascript object>